In [1]:
# Import dependencies

import pandas as pd
import numpy as np
import json 
import requests
import timeit
from citipy import citipy

from datetime import datetime
from config import weather_api_key

In [2]:
# Create a new set of 2,000 random latitudes and longitudes
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

lat_lngs = zip(lats, lngs)
lat_lngs


In [3]:
coordinates = list(lat_lngs)
print (coordinates)

[(-28.8951443139974, -28.602029151682444), (31.466798088191965, 164.74559088682548), (4.137786417168243, 81.01588318242108), (-88.20600768417815, 167.92290015753895), (-33.22882216369382, -154.89860726417825), (-4.930167136814148, 109.97793660051718), (7.864497639723027, 69.9648557991473), (-26.40418932352759, 81.69094741753543), (58.62999242006765, -149.15789357967776), (83.41549735380787, 156.6183736497179), (86.87917292202064, -130.80689047953672), (-54.873223403774354, 14.543584617402843), (5.147226141280569, 92.26750000541261), (-43.813072370082104, 20.502566230261607), (-58.87321348640873, 149.5140836761334), (26.01803058385633, 91.04634323131785), (85.43300519990058, 64.62204276889571), (53.92260741714895, 9.331062109106824), (81.17162893617785, -44.198699967311626), (-69.63035478235206, -109.07098404985399), (-70.4860802285327, -134.72894916446592), (24.5046430615931, 177.01903153683116), (-71.51892098277096, -173.06859316190594), (56.11520911955242, 120.31898728283164), (-37.1

In [10]:
# Get the nearest city using the citipy module
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count 
len(cities)

740

In [11]:
# Perform an API call with the OpenWeatherMap
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [13]:
#Latitude and longitude
#Maximum temperature
#Percent humidity
#Percent cloudiness
#Wind speed
#Weather description (for example, clouds, fog, light rain, clear sky)

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through cities
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | sao joao da barra
Processing Record 2 of Set 1 | severo-kurilsk
Processing Record 3 of Set 1 | matara
Processing Record 4 of Set 1 | bluff
Processing Record 5 of Set 1 | mataura
Processing Record 6 of Set 1 | mlonggo
Processing Record 7 of Set 1 | dhidhdhoo
Processing Record 8 of Set 1 | grand river south east
City not found. Skipping...
Processing Record 9 of Set 1 | sterling
Processing Record 10 of Set 1 | cherskiy
Processing Record 11 of Set 1 | tuktoyaktuk
Processing Record 12 of Set 1 | hermanus
Processing Record 13 of Set 1 | banda aceh
Processing Record 14 of Set 1 | bredasdorp
Processing Record 15 of Set 1 | hobart
Processing Record 16 of Set 1 | barpeta
Processing Record 17 of Set 1 | amderma
City not found. Skipping...
Processing Record 18 of Set 1 | gluckstadt
Processing Record 19 of Set 1 | ilulissat
Processing Record 20 of Set 1 | punta arenas
Processing Record 21 of Set 1 | rikitea


In [14]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Sao Joao Da Barra,BR,-21.6403,-41.0511,71.22,85,3,15.28,clear sky
1,Severo-Kurilsk,RU,50.6789,156.1250,56.79,84,68,8.99,broken clouds
2,Matara,LK,5.9485,80.5353,82.83,75,65,13.35,broken clouds
3,Bluff,NZ,-46.6000,168.3333,50.81,86,100,23.33,overcast clouds
4,Mataura,NZ,-46.1927,168.8643,50.49,92,100,11.23,light rain
5,Mlonggo,ID,-6.5333,110.7000,84.20,58,23,7.49,few clouds
6,Dhidhdhoo,MV,6.8833,73.1000,82.04,82,60,17.87,broken clouds
7,Sterling,US,39.0062,-77.4286,75.43,88,75,3.44,broken clouds
8,Cherskiy,RU,68.7500,161.3000,65.30,53,100,9.37,overcast clouds
9,Tuktoyaktuk,CA,69.4541,-133.0374,75.20,60,75,12.66,broken clouds


In [18]:
# Export the DataFrame as a CSV file
output_data_file = "/Users/codymauc/Desktop/WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="Desktop")